###### 1. На языке Python или С/С++, написать алгоритм (функцию) определения четности целого числа, который будет аналогичен нижеприведенному по функциональности, но отличен по своей сути.
def isEven(value):
    return value%2==0

In [21]:
def isEven(value):
    return value%2==0

def my_isEven(value:int) -> bool:
    """Используя побитовое И"""
    return value&1 == 0   # сравниваем младший разряд

def my_isEven2(value:int) -> bool:
    """Используя побитовые сдвиги"""
    return value == value >> 1 << 1   # заменяем младший разряд нулем путем побитовых сдвигов, сравниваем с исходным значением

###### 2. На языках Python(2.7) и/или С++, написать минимум по 2 класса реализовывающих циклический буфер. 

Объяснить плюсы и минусы каждой реализации.

In [1]:
class CircularBuffer:
    """Циклический буфер через сдвиг списка"""
    def __init__(self,size):
        self._size = size
        self._data = [None for x in range(size)]
    
    
    def add(self,value) -> None:
        """Добавить значение в буфер"""
        self._data.pop()
        self._data.insert(0, value)
  

    def get(self,index:int=0):
        """Извлечь значение из буфера, можно передать параметр и получать предыдущие элементы в порядке их вставки"""
        if 0 <= index <= self._size - 1:  # IndexError
            return self._data[index]
   

    def get_all(self) -> list:
        """Извлечь все элементы из буфера в порядке их вставки"""
        return self._data
 

    def __repr__(self):
        """репрезентация строкой"""
        return self._data.__repr__() + ' (' + str(len(self._data))+' items)'
    

In [2]:
class CircularBuffer2():
    """Циклический буфер через индексацию по списку"""    
    def __init__(self, size):
        self._index = 0
        self._size = size
        self._data = []

        
    def add(self, value) -> None:
        """Добавить значение в буфер"""
        if len(self._data) == self._size:
            self._data[self._index] = value  # Перезапись при полном заполнении
        else:
            self._data.append(value)  # Обновление элемена в списке по индексу
        self._index = (self._index + 1) % self._size  # Приращивание

        
    def get(self,score=False):
        """Извлечь значение из буфера, можно передать параметр и получать предыдущие элементы в порядке их вставки"""
        if score and 0 <= score <= self._size - 1:
            return self._data[(self._size + self._index - 1 - score) % self._size]  # если передан score
        elif score == False:
            return self._data[(self._size + self._index - 1) % self._size]  # если нет score
        else: return  # если score больше длинны списка или отрицательный
    
    
    def get_all(self) -> list:
        """Извлечь все элементы из буфера"""
        return(self._data)
    
    
    def __repr__(self):
        """репрезентация строкой"""
        return self._data.__repr__() + ' (' + str(len(self._data))+' items)'


In [21]:
a = CircularBuffer(100000)
b = CircularBuffer2(100000)

Можем сравнить время используя магическую команду jupiter ноутбука
>%%time

Сначала поместим в буферы 100000 элементов, потом считаем их

In [26]:
%%time
for i in range(100000):
    a.add(i)

CPU times: user 6.06 s, sys: 5.54 ms, total: 6.07 s
Wall time: 6.06 s


In [27]:
%%time
for i in range(100000):
    b.add(i)

CPU times: user 79.6 ms, sys: 2.21 ms, total: 81.8 ms
Wall time: 80.1 ms


In [28]:
%%time
for i in range (100000):
    a.get(i)

CPU times: user 44.5 ms, sys: 2.28 ms, total: 46.8 ms
Wall time: 45.1 ms


In [29]:
%%time
for i in range (100000):
    b.get(i)

CPU times: user 79.1 ms, sys: 2.45 ms, total: 81.6 ms
Wall time: 79.8 ms


1) Как и ожидалось, на запись 1 буфер в разы медленнее, "двигать" элементы путем pop() и insert() очень дорого. На чтение нет никаких сложных операций, чтение основывается только на индексе.

2) 2 буфер на запись использует только append() и работу с индексами. На чтение он медленнее, так как использует большее количество арифметических операций, чем буфер 1.

###### 3. На языке Python или С/С++, написать функцию, которая быстрее всего (по процессорным тикам) отсортирует данный ей массив чисел.

Массив может быть любого размера со случайным порядком чисел (в том числе и отсортированным).
Объяснить почему вы считаете, что функция соответствует заданным критериям.

Я решил использовать сортировку слиянием.

Сортировка сортировке рознь! Все зависит от исходного массива(от количества элементов в нем, от их веса и порядка). Основным преимуществом сортировки слиянием является надежное время выполнения алгоритма и его эффективность при сортировке больших массивов. Временная сложность равна O(N logN) не зависит от порядка элементов в массиве. В отличие от Quick Sort, которая чаще бывает быстрее, в сортировке слиянием нет каких-либо неудачных решений, которые приводят к плохому времени выполнения. Однако на маленьких массивах слияние будет неэффективно.

Для удобства и оптимизации в 2001 году была реализована сортировка Timsort, она сортирует вставкой массивы до 64 элементов, либо разбивает большой массив на подмассивы и сортирует их вставкой, подмассивы собираются в единый массив с помощью модифицированной сортировки слиянием. Для ее вызова:

>list.sort()

>sorted(list)

In [47]:
def mergeSort(arr: list) -> None:
    """Сортировка слиянием"""
    if len(arr) > 1:  # Пока в массиве больше одного элемента будем его разделять
        mid = len(arr)//2
        # Выделяем две примерно равные части массива
        L = arr[:mid]
        R = arr[mid:]
        # Продолжаем делить пока не получим по 1 элементу в массиве
        mergeSort(L)
        mergeSort(R)

        left_index = right_index = sorted_index = 0
        
        # Процесс склеивания
        while left_index < len(L) and right_index < len(R):  # Пока имеем и в L и в R элементы которые еще не перебрали
            if L[left_index] < R[right_index]:
                arr[sorted_index] = L[left_index]
                left_index += 1
            else:
                arr[sorted_index] = R[right_index]
                right_index += 1
            sorted_index += 1

        while left_index < len(L):  # Когда R уже перебрали, оставшиеся из L добавляем
            arr[sorted_index] = L[left_index]
            left_index += 1
            sorted_index += 1

        while right_index < len(R):  # Когда L уже перебрали, оставшиеся из R добавляем
            arr[sorted_index] = R[right_index]
            right_index += 1
            sorted_index += 1
